In [ ]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [ ]:
import glob
'''
all_files = glob.glob("/content/drive/My Drive/dataset/implementation/4 (extra prev removed).csv")
frame = pd.DataFrame()
list_ = []
for file_ in all_files:
    df = pd.read_csv(file_,index_col=None, sep=',', header=None, names=['text','types','to_delete'])
    list_.append(df.iloc[:,:-1])
df = pd.concat(list_)
'''
df = pd.read_csv('Train(5333).csv')
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)
print('No. of unique classes',len(set(df.iloc[:,1])))


Shape of dataset  (5333, 2)
Index(['Text', 'Label'], dtype='object')
No. of unique classes 5


In [ ]:
df = pd.read_csv('Train(5333).csv')
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)
print('No. of unique classes',len(set(df.iloc[:,1])))


In [ ]:
df1 = pd.read_csv('Test(1334).csv')
df1 = df1.dropna()
df1 = df1.reset_index(drop=True)
print('Shape of dataset ',df1.shape)
print(df1.columns)
print('No. of unique classes',len(set(df1.iloc[:,1])))


Shape of dataset  (1334, 2)
Index(['Text', 'Label'], dtype='object')
No. of unique classes 5


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 4.csv to 4.csv


In [ ]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.head()

,WP$,PRP$,VBG,EX,RBR,$,CC,TO,NNPS,JJ,CD,RBS,WP,WRB,NNP,VB,VBP,UH,VBN,VBZ,IN,RP,VBD,MD,NNS,JJR,DT,NN,JJS,RB,WDT,FW,PRP
0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,5,1,0,0,0,0,0,3,0,0,0,0,0,1,0,3,0,0,3,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,2,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,0


In [ ]:
df['token_length'] = [len(x.split(" ")) for x in df.Text]
max(df.token_length)

65

In [ ]:
df.loc[df.token_length.idxmax(),'Text']

'russia publish info pre clinical clinical trials covid 19 vaccine soon would someone vaccinate child chicken pox vaccine child chicken pox already measles that treatment let s take science would trust vaccine donald trump promoting he s promoting chloraquinetreatment wants locking ive seen mask worn yorkshire 7 week uk heading back lockdown next mo watched tamil moviepremiere 2020 bigil 164m impressions watched tamil movie lockdown'

In [ ]:

df1['token_length'] = [len(x.split(" ")) for x in df1.Text]
max(df1.token_length)

37

In [ ]:
# cleaning

In [ ]:
df.Text[:10]

0             100 188513 deaths covid covid 6 deaths w
1    100 dav ive got post covid19 maladies include ...
2                  100 deaths covid covid 6 deaths peo
3             1000 per day read fine seeing want say h
4    10000 people preexisting conditions asthmahigh...
5    10k means 10k 100 healthy people died covid 18...
6                       10k number roundly rejected as
7    12 bradykinin peptide lower blood pressure inc...
8    12 dont tweet often something happened complet...
9     12 dont understand data amp sciencei get youre a
Name: Text, dtype: object

**original imbalance data**


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

tvec = TfidfVectorizer( max_features = 10000, ngram_range=(1, 3))
lr =LogisticRegression(random_state=0)

In [ ]:
X = np.asanyarray(df['Text'])
X = X.reshape(-1, 1)
X.shape

(5333, 1)

In [ ]:
X1 = np.asanyarray(df1['Text'])
X1 = X1.reshape(-1, 1)
X1.shape

(1334, 1)

In [ ]:
Y = np.asanyarray(df['Label'])
Y = Y.reshape(-1, 1)
Y.shape

(5333, 1)

In [ ]:
Y1 = np.asanyarray(df1['Label'])
Y1 = Y1.reshape(-1, 1)
Y1.shape

(1334, 1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

def lr_cv(pipeline, average_method):
    
    X_train=df['Text']
    X_test=df1['Text']
    y_train=df['Label']
    y_test =df1['Label']

    accuracy = []
    precision = []
    recall = []
    f1 = []
    lr_fit = pipeline.fit(X_train, y_train)
    prediction = lr_fit.predict(X_test)
    scores = lr_fit.score(X_test,y_test)
        
    accuracy.append(scores * 100)
    precision.append(precision_score(y_test, prediction, average=average_method)*100)
    print('              negative    neutral     positive')
    print('precision:',precision_score(y_test, prediction, average=None))
    recall.append(recall_score(y_test, prediction, average=average_method)*100)
    print('recall:   ',recall_score(y_test, prediction, average=None))
    f1.append(f1_score(y_test, prediction, average=average_method)*100)
    print('f1 score: ',f1_score(y_test, prediction, average=None))
    print('-'*50)

    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))
    from sklearn.metrics import confusion_matrix,classification_report
    print(confusion_matrix(y_test, prediction))
    print(classification_report(y_test, prediction))

In [ ]:
from sklearn.pipeline import Pipeline

original_pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', lr)
])

In [ ]:
X =  df#features
y = y.values #target
print (X.shape)
print(y.shape)

(6667, 35)
(6667,)


In [ ]:
X = np.asanyarray(X)
X = X.reshape((6666,1))


In [ ]:
print (X.shape)

(6666, 1)


In [ ]:

lr_cv(original_pipeline, 'macro')


              negative    neutral     positive
precision: [0.58762887 0.77536232 0.92114695 0.97241379 0.96688742]
recall:    [0.59685864 0.79455446 0.92779783 0.87037037 0.97333333]
f1 score:  [0.59220779 0.78484108 0.92446043 0.91856678 0.97009967]
--------------------------------------------------
accuracy: 84.33% (+/- 0.00%)
precision: 84.47% (+/- 0.00%)
recall: 83.26% (+/- 0.00%)
f1 score: 83.80% (+/- 0.00%)
[[114  58  13   1   5]
 [ 73 321   7   1   2]
 [  0  16 257   2   2]
 [  5  13   2 141   1]
 [  2   6   0   0 292]]
              precision    recall  f1-score   support

 health risk       0.59      0.60      0.59       191
  prevention       0.78      0.79      0.78       404
    symptoms       0.92      0.93      0.92       277
transmission       0.97      0.87      0.92       162
   treatment       0.97      0.97      0.97       300

    accuracy                           0.84      1334
   macro avg       0.84      0.83      0.84      1334
weighted avg       0.85      0.84

**RandomOverSampler**

In [ ]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

ROS_pipeline = make_pipeline(tvec,RandomOverSampler(random_state=777),lr)
SMOTE_pipeline = make_pipeline(tvec,SMOTE(random_state=777),lr)
ADASYN_pipeline = make_pipeline(tvec,ADASYN(ratio='minority',random_state=777),lr)

In [ ]:
'''
sent1 = "I love dogs"
sent2 = "I don't like dogs"
sent3 = "I adore cats"
sent4 = "I hate spiders"
sent5 = "I like dogs"
testing_text = pd.Series([sent1, sent2, sent3, sent4, sent5])
testing_target = pd.Series([1,0,1,0,1])
'''

In [ ]:
'''
tv = TfidfVectorizer(stop_words=None, max_features=100000)
testing_tfidf = tv.fit_transform(testing_text)
'''

In [ ]:
'''
ros = RandomOverSampler(random_state=777)
X_ROS, y_ROS = ros.fit_sample(testing_tfidf, testing_target)
'''

In [ ]:
'''
pd.DataFrame(testing_tfidf.todense(), columns=tv.get_feature_names())
'''

In [ ]:
'''
pd.DataFrame(X_ROS.todense(), columns=tv.get_feature_names())
'''

In [ ]:
#y_ROS

array([1, 0, 1, 0, 1, 0])

In [ ]:
lr_cv( ROS_pipeline, 'macro')


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              negative    neutral     positive
precision: [0.56870229 0.87463557 0.93140794 0.96052632 0.97666667]
recall:    [0.78010471 0.74257426 0.93140794 0.90123457 0.97666667]
f1 score:  [0.65783664 0.80321285 0.93140794 0.92993631 0.97666667]
--------------------------------------------------
accuracy: 85.91% (+/- 0.00%)
precision: 86.24% (+/- 0.00%)
recall: 86.64% (+/- 0.00%)
f1 score: 85.98% (+/- 0.00%)
[[149  28   9   2   3]
 [ 92 300   8   2   2]
 [  8   7 258   2   2]
 [  9   5   2 146   0]
 [  4   3   0   0 293]]
              precision    recall  f1-score   support

 health risk       0.57      0.78      0.66       191
  prevention       0.87      0.74      0.80       404
    symptoms       0.93      0.93      0.93       277
transmission       0.96      0.90      0.93       162
   treatment       0.98      0.98      0.98       300

    accuracy                           0.86      1334
   macro avg       0.86      0.87      0.86      1334
weighted avg       0.88      0.86

In [ ]:
df.text

0        large gathering limits placed sacramento pray...
1        novel coronavirus infection conducted test ci...
2        recent mania overtreating high blood pressure...
3        yll would blame bron fires cali global warmin...
4       07222020 indian slum contained possible covid ...
                              ...                        
6653         zero hedge reports approved covid 19 vaccine
6654    zimbabwean businessman strive masiyiwa donated...
6655    zones plasma therapy critical covid patients h...
6656    zoom fatigue new normal uncertain times overco...
6657    zoom fatigue plays important role affecting me...
Name: text, Length: 6658, dtype: object

**SMOTE**

In [ ]:
smt = SMOTE(random_state=777, k_neighbors=1)
X_SMOTE, y_SMOTE = smt.fit_sample(testing_tfidf, testing_target)
pd.DataFrame(X_SMOTE.todense(), columns=tv.get_feature_names())

NameError: ignored

In [ ]:

pd.DataFrame(X_SMOTE.todense()[y_SMOTE == 0], columns=tv.get_feature_names())


In [ ]:
lr_cv(SMOTE_pipeline, 'macro')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              negative    neutral     positive
precision: [0.57258065 0.86039886 0.90877193 0.97315436 0.97009967]
recall:    [0.7434555  0.74752475 0.93501805 0.89506173 0.97333333]
f1 score:  [0.64692483 0.8        0.92170819 0.93247588 0.97171381]
--------------------------------------------------
accuracy: 85.46% (+/- 0.00%)
precision: 85.70% (+/- 0.00%)
recall: 85.89% (+/- 0.00%)
f1 score: 85.46% (+/- 0.00%)
[[142  29  14   1   5]
 [ 89 302  10   1   2]
 [  5   9 259   2   2]
 [  8   7   2 145   0]
 [  4   4   0   0 292]]
              precision    recall  f1-score   support

 health risk       0.57      0.74      0.65       191
  prevention       0.86      0.75      0.80       404
    symptoms       0.91      0.94      0.92       277
transmission       0.97      0.90      0.93       162
   treatment       0.97      0.97      0.97       300

    accuracy                           0.85      1334
   macro avg       0.86      0.86      0.85      1334
weighted avg       0.87      0.85

**ADASYN**

In [ ]:
lr_cv(  ADASYN_pipeline, 'macro')


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


              negative    neutral     positive
precision: [0.59685864 0.78869779 0.92114695 0.95512821 0.97009967]
recall:    [0.59685864 0.79455446 0.92779783 0.91975309 0.97333333]
f1 score:  [0.59685864 0.79161529 0.92446043 0.93710692 0.97171381]
--------------------------------------------------
accuracy: 84.93% (+/- 0.00%)
precision: 84.64% (+/- 0.00%)
recall: 84.25% (+/- 0.00%)
f1 score: 84.44% (+/- 0.00%)
[[114  56  14   2   5]
 [ 71 321   7   3   2]
 [  0  16 257   2   2]
 [  4   8   1 149   0]
 [  2   6   0   0 292]]
              precision    recall  f1-score   support

 health risk       0.60      0.60      0.60       191
  prevention       0.79      0.79      0.79       404
    symptoms       0.92      0.93      0.92       277
transmission       0.96      0.92      0.94       162
   treatment       0.97      0.97      0.97       300

    accuracy                           0.85      1334
   macro avg       0.85      0.84      0.84      1334
weighted avg       0.85      0.85

In [ ]:
SMOTE CF
[[149  28   7   3   8]
 [ 90 299  12   6   2]
 [  2  10 263   2   2]
 [  2   5   2 151   0]
 [  3   4   1   2 278]]